# Preprocessing

In [38]:
import numpy as np
import pandas as pd
datacols = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","attack", "last_flag"]
selected_features=['src_bytes',
 'dst_bytes',
 'logged_in',
 'count',
 'srv_count',
 'dst_host_srv_count',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_serror_rate',
 'service']
output=['attack']

In [39]:
dfkdd_train = pd.read_table("KDDTrain+.txt", sep=",", names=datacols)
dfkdd_test = pd.read_table("KDDTest+.txt", sep=",", names=datacols)

In [40]:
X_train=dfkdd_train[selected_features]
X_test=dfkdd_test[selected_features]
Y_train=dfkdd_train['attack']
Y_test=dfkdd_test['attack']

In [41]:
mapping = {'ipsweep': 'Probe','satan': 'Probe','nmap': 'Probe','portsweep': 'Probe','saint': 'Probe','mscan': 'Probe',
        'teardrop': 'DoS','pod': 'DoS','land': 'DoS','back': 'DoS','neptune': 'DoS','smurf': 'DoS','mailbomb': 'DoS',
        'udpstorm': 'DoS','apache2': 'DoS','processtable': 'DoS',
        'perl': 'U2R','loadmodule': 'U2R','rootkit': 'U2R','buffer_overflow': 'U2R','xterm': 'U2R','ps': 'U2R',
        'sqlattack': 'U2R','httptunnel': 'U2R',
        'ftp_write': 'R2L','phf': 'R2L','guess_passwd': 'R2L','warezmaster': 'R2L','warezclient': 'R2L','imap': 'R2L',
        'spy': 'R2L','multihop': 'R2L','named': 'R2L','snmpguess': 'R2L','worm': 'R2L','snmpgetattack': 'R2L',
        'xsnoop': 'R2L','xlock': 'R2L','sendmail': 'R2L',
        'normal': 'Normal'
        }

In [42]:
Y_train=Y_train.apply(lambda v: mapping[v])
Y_test=Y_test.apply(lambda v: mapping[v])

In [43]:
import imblearn
from imblearn.over_sampling import RandomOverSampler 
from collections import Counter
ros = RandomOverSampler(random_state=42)
X_res,Y_res = ros.fit_sample(X_train, Y_train)
print('Original dataset shape {}'.format(Counter(Y_train)))
print('Resampled dataset shape {}'.format(Counter(Y_res)))

Using TensorFlow backend.
/home/sudhamsa/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sudhamsa/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sudhamsa/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sudhamsa/.local/lib/python3.6/site-packages/tensorflo

Original dataset shape Counter({'Normal': 67343, 'DoS': 45927, 'Probe': 11656, 'R2L': 995, 'U2R': 52})
Resampled dataset shape Counter({'Normal': 67343, 'DoS': 67343, 'R2L': 67343, 'Probe': 67343, 'U2R': 67343})


In [44]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
cols = X_train.select_dtypes(include=['float64','int64']).columns
Stscaled_Xtrain=pd.DataFrame(scaler.fit_transform(X_res.select_dtypes(include=['float64','int64'])),columns=cols)
Stscaled_Xtest=pd.DataFrame(scaler.fit_transform(X_test.select_dtypes(include=['float64','int64'])),columns=cols)

In [45]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
cols = X_train.select_dtypes(include=['float64','int64']).columns
scaler.fit(X_train.select_dtypes(include=['float64','int64']))
MinMax_Xtrain=pd.DataFrame(scaler.transform(X_res.select_dtypes(include=['float64','int64'])),columns=cols)
MinMax_Xtest=pd.DataFrame(scaler.transform(X_test.select_dtypes(include=['float64','int64'])),columns=cols)

In [46]:
Y_train=Y_res.apply(lambda v:0 if(v=='Normal') else 1)
Y_test=Y_test.apply(lambda v:0 if(v=='Normal') else 1)

In [47]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
catXres=X_res[['service']].copy()
catXtest=X_test[['service']].copy()
enc.fit(catXres)
cols=enc.categories_[0]
enctrain=enc.transform(catXres).toarray()
enctest=enc.transform(catXtest).toarray()
cattrain=pd.DataFrame(enctrain,columns=cols)
cattest=pd.DataFrame(enctest,columns=cols)

In [48]:
X_train=pd.concat([MinMax_Xtrain,cattrain],axis=1)
X_test=pd.concat([MinMax_Xtest,cattest],axis=1)

In [49]:
from sklearn.utils import shuffle
X_train_, Y_train_ = shuffle(X_train,Y_train, random_state=0)

In [50]:
X_train

,src_bytes,dst_bytes,logged_in,count,srv_count,dst_host_srv_count,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate,IRC,...,telnet,tftp_u,tim_i,time,urh_i,urp_i,uucp,uucp_path,vmnet,whois
0,3.558064e-07,0.000000e+00,0.0,0.003914,0.003914,0.098039,0.03,0.17,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.057999e-07,0.000000e+00,0.0,0.025440,0.001957,0.003922,0.60,0.88,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000e+00,0.000000e+00,0.0,0.240705,0.011742,0.101961,0.05,0.00,1.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.681203e-07,6.223962e-06,1.0,0.009785,0.009785,1.000000,0.00,0.03,0.03,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.442067e-07,3.206260e-07,1.0,0.058708,0.062622,1.000000,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336710,0.000000e+00,4.302496e-06,1.0,0.001957,0.001957,0.160784,0.00,1.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
336711,1.129740e-06,2.085596e-06,1.0,0.001957,0.001957,0.031373,0.00,0.12,0.00,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
336712,0.000000e+00,4.449068e-06,1.0,0.003914,0.003914,0.007843,0.00,1.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
336713,0.000000e+00,1.581755e-06,1.0,0.005871,0.005871,0.329412,0.00,1.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
Y_train

0         0
1         0
2         1
3         0
4         0
         ..
336710    1
336711    1
336712    1
336713    1
336714    1
Name: attack, Length: 336715, dtype: int64

In [52]:
X_test

,src_bytes,dst_bytes,logged_in,count,srv_count,dst_host_srv_count,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate,IRC,...,telnet,tftp_u,tim_i,time,urh_i,urp_i,uucp,uucp_path,vmnet,whois
0,0.000000e+00,0.000000e+00,0.0,0.448141,0.019569,0.039216,0.06,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000e+00,0.000000e+00,0.0,0.266145,0.001957,0.003922,0.06,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9.408217e-06,0.000000e+00,0.0,0.001957,0.001957,0.337255,0.04,0.61,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.449313e-08,0.000000e+00,0.0,0.001957,0.127202,0.223529,0.00,1.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000e+00,1.145093e-08,0.0,0.001957,0.015656,0.337255,0.17,0.03,0.00,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22539,5.753774e-07,2.542106e-07,1.0,0.001957,0.001957,0.552941,0.06,0.01,0.01,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22540,2.297162e-07,7.160648e-07,1.0,0.003914,0.021526,1.000000,0.00,0.01,0.01,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22541,3.952277e-05,6.346868e-06,1.0,0.009785,0.019569,1.000000,0.00,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22542,3.043558e-08,3.206260e-08,0.0,0.007828,0.011742,0.988235,0.01,0.00,0.00,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
Y_test

0        1
1        1
2        0
3        1
4        1
        ..
22539    0
22540    0
22541    1
22542    0
22543    1
Name: attack, Length: 22544, dtype: int64

# Neural Network Architecture

In [54]:
def forward_prop(params,X,Y,dim):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss. It receives a set of parameters that must be
    rolled-back into the corresponding weights and biases.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """
     # Neural network architecture
    n_inputs = X.shape[1]
    n_hidden = 20
    n_classes = 2

    # Roll-back the weights and biases
    W1 = params[0:(n_inputs*n_hidden)].reshape((n_inputs,n_hidden))
    b1 = params[(n_inputs*n_hidden):(n_inputs*n_hidden+n_hidden)].reshape((n_hidden,))
    W2 = params[(n_inputs*n_hidden+n_hidden):(n_inputs*n_hidden+n_hidden+n_hidden*2)].reshape((n_hidden,n_classes))
    b2 = params[(n_inputs*n_hidden+n_hidden+n_hidden*2):dim].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
    # Compute for the negative log likelihood
    N = Y.shape[0] # Number of samples
#     print(probs,"no error")
#     print(probs[range(N)],"pi")
    corect_logprobs = -np.log(probs[range(N), Y])
#     print(corect_logprobs,"no error1")
    loss = np.sum(corect_logprobs) / N
#     print("no err")
    return loss

In [55]:
import numpy as np
def predict(X, pos,dim):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    X: numpy.ndarray
        Input Iris dataset
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """

    # Neural network architecture
    n_inputs = X.shape[1]
    n_hidden = 20
    n_classes = 2

    # Roll-back the weights and biases
    W1 = pos[0:(n_inputs*n_hidden)].reshape((n_inputs,n_hidden))
    b1 = pos[(n_inputs*n_hidden):(n_inputs*n_hidden+n_hidden)].reshape((n_hidden,))
    W2 = pos[(n_inputs*n_hidden+n_hidden):(n_inputs*n_hidden+n_hidden+n_hidden*2)].reshape((n_hidden,n_classes))
    b2 = pos[(n_inputs*n_hidden+n_hidden+n_hidden*2):dim].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    z2 = a1.dot(W2) + b2 # Pre-activation in Layer 2
    logits = z2          # Logits for Layer 2
    
    y_pred = np.argmax(logits, axis=1)
    return y_pred

# Optimization Using Nature Inspired Algorithms

In [57]:
dimensions=79*20+20*2+20+2
lb=-10
ub=10
Popsize=1000
iters=20

In [ ]:
from PSO import PSO
from GWO import GWO
from MVO import MVO
from MFO import MFO

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

import matplotlib.pyplot as plt

def optimize(optimizer,trainInput,trainOutput,testInput,testOutput):
    s=optimizer(forward_prop,lb,ub,dimensions,Popsize,iters,trainInput,trainOutput)
    y_pred=predict(testInput,s.bestIndividual,dimensions)
    accuracy=(y_pred==testOutput).mean()
    results=precision_recall_fscore_support(y, y_predi, average='macro')
    ConfMatrix=confusion_matrix(y, predi)
    plt.plot(s.convergence)
    plt.show()
    return {'confusion_matrix':ConfMatrix, 'Precision,recall,fscore':results, 'accuracy':accuray}

In [ ]:
gwo=optimize(GWO,X_train.to_numpy(),Y_train.to_numpy(),X_test.to_numpy(),Y_test.to_numpy())

GWO is optimizing  "forward_prop"
['At iteration 0 the best fitness is 1.2532160493617042']
['At iteration 1 the best fitness is 1.1858637023605916']
['At iteration 2 the best fitness is 0.7431253167576644']
['At iteration 3 the best fitness is 0.7431253167576644']
['At iteration 4 the best fitness is 0.6209226021901431']
['At iteration 5 the best fitness is 0.5418344147656444']
['At iteration 6 the best fitness is 0.4146754484352776']
['At iteration 7 the best fitness is 0.41162224471845243']
['At iteration 8 the best fitness is 0.34607267941649']
['At iteration 9 the best fitness is 0.28187283902479204']
['At iteration 10 the best fitness is 0.27066625125555815']
['At iteration 11 the best fitness is 0.23201902196168508']
['At iteration 12 the best fitness is 0.22199429626021142']
['At iteration 13 the best fitness is 0.20903524434369922']
['At iteration 14 the best fitness is 0.20903524434369922']


In [ ]:
pso=optimize(PSO,X_train.to_numpy(),Y_train.to_numpy(),Y_train.to_numpy(),Y_test.to_numpy())
mvo=optimize(MVO,X_train.to_numpy(),Y_train.to_numpy(),Y_train.to_numpy(),Y_test.to_numpy())
mfo=optimize(MFO,X_train.to_numpy(),Y_train.to_numpy(),Y_train.to_numpy(),Y_test.to_numpy())

In [74]:
gwo['Precision,recall,fscore']

(0.9403647965131454, 0.9114347445168762, 0.9249748291912674, None)

In [73]:
print(gwo['accuracy'],'Accuracy')

0.9535393433615966 Accuracy


In [68]:
gwo['confusion_matrix']

array([[ 56653,  10690],
       [  4954, 264418]])